## Preparing Data, Device and Modules

In [2]:
# importing libraries

from brainflow.board_shim import BoardShim, BrainFlowInputParams, LogLevels, BoardIds
from brainflow.data_filter import DataFilter, WindowOperations, DetrendOperations, FilterTypes
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import time

In [3]:
# enable BrainFlow logger

BoardShim.enable_dev_board_logger()

In [4]:
# input parameter port and timeout

params = BrainFlowInputParams()
params.serial_port = '/dev/ttyACM0'
params.timeout = 15

In [5]:
# board_id (ganglion) detail

board_id = BoardIds.GANGLION_BOARD.value
board_descr = BoardShim.get_board_descr(board_id)
sampling_rate = int(board_descr['sampling_rate'])
board = BoardShim(board_id, params)

In [6]:
nfft = DataFilter.get_nearest_power_of_two(sampling_rate)

In [7]:
eeg_channels = board_descr['eeg_channels']
eeg_channel1 = eeg_channels[0]
eeg_channel2 = eeg_channels[1]
eeg_channel3 = eeg_channels[2]
eeg_channel4 = eeg_channels[3]

In [8]:
data = pd.read_csv('/home/bimanjaya/learner/TA/brainflow/EEG-projects/src/data/mindful.csv')
data = data.transpose()
data = data.to_numpy()

In [9]:
data_bb = data

In [10]:
delay = 5
total_iterate = data_bb.shape[1] // (sampling_rate*delay)
total_iterate

36

In [11]:
# eeg 1

dict = {'alpha1':[],'alpha2':[],'alpha3':[],'alpha4':[],
        'beta1':[],'beta2':[],'beta3':[],'beta4':[]}
df = pd.DataFrame(dict)

for x in range(total_iterate):
    ch1_iter = data_bb[eeg_channel1][(x*1000):((x+1)*1000)]
    ch2_iter = data_bb[eeg_channel2][(x*1000):((x+1)*1000)]
    ch3_iter = data_bb[eeg_channel3][(x*1000):((x+1)*1000)]
    ch4_iter = data_bb[eeg_channel4][(x*1000):((x+1)*1000)]
    
    DataFilter.detrend(ch1_iter, DetrendOperations.LINEAR.value)
    DataFilter.detrend(ch2_iter, DetrendOperations.LINEAR.value)
    DataFilter.detrend(ch3_iter, DetrendOperations.LINEAR.value)
    DataFilter.detrend(ch4_iter, DetrendOperations.LINEAR.value)
    
    DataFilter.perform_bandpass(ch1_iter, BoardShim.get_sampling_rate(board_id), 5.0, 35.0, 4,
                                        FilterTypes.BUTTERWORTH.value, 0)
    DataFilter.perform_bandpass(ch2_iter, BoardShim.get_sampling_rate(board_id), 5.0, 35.0, 4,
                                        FilterTypes.BUTTERWORTH.value, 0)
    DataFilter.perform_bandpass(ch3_iter, BoardShim.get_sampling_rate(board_id), 5.0, 35.0, 4,
                                        FilterTypes.BUTTERWORTH.value, 0)
    DataFilter.perform_bandpass(ch4_iter, BoardShim.get_sampling_rate(board_id), 5.0, 35.0, 4,
                                        FilterTypes.BUTTERWORTH.value, 0)
    
    psd1 = DataFilter.get_psd_welch(ch1_iter, nfft, nfft // 2, sampling_rate,
                                   WindowOperations.HAMMING.value)
    psd2 = DataFilter.get_psd_welch(ch2_iter, nfft, nfft // 2, sampling_rate,
                                   WindowOperations.HAMMING.value)
    psd3 = DataFilter.get_psd_welch(ch3_iter, nfft, nfft // 2, sampling_rate,
                                   WindowOperations.HAMMING.value)
    psd4 = DataFilter.get_psd_welch(ch4_iter, nfft, nfft // 2, sampling_rate,
                                   WindowOperations.HAMMING.value)
    
    band_power_total1 = DataFilter.get_band_power(psd1, psd1[1][0], psd1[1][-1])
    band_power_total2 = DataFilter.get_band_power(psd2, psd2[1][0], psd2[1][-1])
    band_power_total3 = DataFilter.get_band_power(psd3, psd3[1][0], psd3[1][-1])
    band_power_total4 = DataFilter.get_band_power(psd4, psd4[1][0], psd4[1][-1])
    
    band_power_alpha1 = DataFilter.get_band_power(psd1, 8.0, 13.0)
    band_power_alpha2 = DataFilter.get_band_power(psd2, 8.0, 13.0)
    band_power_alpha3 = DataFilter.get_band_power(psd3, 8.0, 13.0)
    band_power_alpha4 = DataFilter.get_band_power(psd4, 8.0, 13.0)
    
    alpha_relative1 = band_power_alpha1/band_power_total1
    alpha_relative2 = band_power_alpha2/band_power_total2
    alpha_relative3 = band_power_alpha3/band_power_total3
    alpha_relative4 = band_power_alpha4/band_power_total4
    
    band_power_beta1 = DataFilter.get_band_power(psd1, 13.0, 32.0)
    band_power_beta2 = DataFilter.get_band_power(psd2, 13.0, 32.0)
    band_power_beta3 = DataFilter.get_band_power(psd3, 13.0, 32.0)
    band_power_beta4 = DataFilter.get_band_power(psd4, 13.0, 32.0)
    
    beta_relative1 = band_power_beta1/band_power_total1
    beta_relative2 = band_power_beta2/band_power_total2
    beta_relative3 = band_power_beta3/band_power_total3
    beta_relative4 = band_power_beta4/band_power_total4
    
    dict1 = {'alpha1':[alpha_relative1],'alpha2':[alpha_relative2],
             'alpha3':[alpha_relative3],'alpha4':[alpha_relative4],
             'beta1':[beta_relative1],'beta2':[beta_relative2],
             'beta3':[beta_relative3],'beta4':[beta_relative4]}
    
    df2 = pd.DataFrame(dict1)
    df = pd.concat([df,df2],ignore_index=True)
    # df.reset_index()

display(df.head())

,alpha1,alpha2,alpha3,alpha4,beta1,beta2,beta3,beta4
0,0.215083,0.235176,0.219503,0.231384,0.344209,0.247658,0.261818,0.284674
1,0.214565,0.272064,0.251035,0.201310,0.344056,0.304483,0.240963,0.229385
2,0.214155,0.224876,0.220907,0.210469,0.343213,0.255480,0.247330,0.220121
3,0.213462,0.237184,0.213580,0.210826,0.340411,0.266235,0.227554,0.238421
4,0.211709,0.241091,0.225107,0.216737,0.339607,0.249541,0.219162,0.235900


In [12]:
from joblib import dump,load

In [13]:
loaded = load('/home/bimanjaya/learner/TA/brainflow/EEG-projects/src/model1.joblib') 

In [22]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report

In [20]:
data1 = df.iloc[1].tolist()

In [25]:
loaded.predict([data1])

/home/bimanjaya/.conda/envs/workspace1/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


array([1])